# Collisions

In [1]:
import numpy as np
from pygraphics import graphics as gp
import time

### Basic Functions

In [2]:
def distancia(A,B):
    p1 = A.getCenter()
    p2 = B.getCenter()
    return np.sqrt((p1.x-p2.x)**2+(p1.y-p2.y)**2)

def angle(A,B):
    p1 = A.getCenter()
    p2 = B.getCenter()
    return np.arctan((p2.y-p1.y)/(p1.x-p2.x))

def transformar(i,j,theta):
    
    A = np.array([Vx[i], Vy[i]])
    B = np.array([Vx[j], Vy[j]])
    M = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    Mi = (1/np.linalg.det(M))*np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta) ,np.cos(theta)]])
    Ap = np.dot(M,A)
    Bp = np.dot(M,B)
    
    if np.dot(A,B) < 0:
        Ap[0] = -Ap[0]
        Bp[0] = -Bp[0]
    else:
        if np.linalg.norm(A) > np.linalg.norm(B): 
            Ap[0] = -Ap[0]
            Bp[1] = -Bp[1]
        else:
            Ap[1] = -Ap[1]
            Bp[0] = -Bp[0]
        
    A = np.dot(Mi,Ap)
    B = np.dot(Mi,Bp)

    Vx[i], Vy[i] =  A[0], -A[1]
    Vx[j], Vy[j] =  B[0], -B[1]

### General Set Up

In [3]:
radi = 20.
fI = 0.9
velo = 500.
g = 200

pos1 = [400,490]
pos2 = [800,510]
v1 = [2,0.5]
v2 = [-1,-0.1]

numBoles = 2
boles = []

t0 = time.time()
td = time.time()
interval = 1/60.
itv = interval*2000

Ax = []
Rx = []
Ay = []
Vx = [v1[0]*velo,v2[0]*velo]
Vy = [v1[1]*velo,v2[1]*velo]
#Vx = []
#Vy = []
Ry = []
canvi = []
colisio = []

for i in range(numBoles):
    ax = 0
    ay = g

    vx = Vx[i]
    vy = Vy[i]
    #vx = ax/itv + (np.random.rand()-0.5)*velo
    #vy = ay/itv + (np.random.rand()-0.5)*velo
    
    dx = vx/itv
    dy = vy/itv
    
    Ax.append(ax)
    Ay.append(ay)
    Vx.append(vx)
    Vy.append(vy)
    Rx.append(dx)
    Ry.append(dy)
    canvi.append(0)
    colisio.append(0)

### Main Loop

In [4]:
win = gp.GraphWin("Field", 1000, 1000)
win.setBackground("black")

for i in range(numBoles):    
    bola = gp.Circle(gp.Point(100+800*np.random.rand(),100+800*np.random.rand()), radi)
    bola.setFill("orange")
    bola.setOutline("white")
    bola.draw(win)
    boles.append(bola)
    

while time.time()-t0 < 10:
    if time.time()-td > interval:
        td = time.time()
        
        for i in range(len(boles)):
            for j in range(i+1,len(boles)):
                dist = distancia(boles[i], boles[j])
                if dist < radi*2:
                    if colisio[i] == 0:
                        transformar(i,j, angle(boles[i],boles[j]))
                        colisio[i] = 1
                else:
                    colisio[i] = 0
            centre = boles[i].getCenter()
            
            Vy[i] = Vy[i]+Ay[i]/itv
            
            if centre.x < radi or centre.x > 1000-radi:
                Vx[i] = -Vx[i]
                
            if centre.y < radi or centre.y > 1000-radi:
                if canvi[i] == 0:
                    Vy[i] = -Vy[i]*fI
                    canvi[i] = 1
            else:
                canvi[i] = 0

            Rx[i] = Vx[i]/itv
            Ry[i] = Vy[i]/itv
            
            boles[i].move(Rx[i], Ry[i])
        
win.close()